In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.preprocessing import StandardScaler

In [3]:
df_data = pd.DataFrame({'col1': [1, 2, 3], 'col2': [4, 5, 6]})

In [6]:
result = StandardScaler().fit_transform(df_data)
pd.DataFrame(result, columns=['col1', 'col2'])

,col1,col2
0,-1.224745,-1.224745
1,0.000000,0.000000
2,1.224745,1.224745
